In [149]:
from nltk.tokenize import sent_tokenize
import numpy as np
import os
import time

In [150]:
file_path = './hdt-tagged/hdt-1-10000-train.tags'
test_file_path = './hdt-tagged/hdt-10001-12000-test.tags'


def parse_file(file_path):
    with open(file_path, "r") as file:
        read_data = file.read()
    # We want a numpy array because it's computational faster. First we lower the string and then we split it.
    lines = np.array(read_data.lower().split('\n'))
    # Let's split the lines again this time the seperator is \t
    data2d = list(map(lambda d: d.split('\t'),lines))
    # Filter out each single column row
    data2d = np.array(list(filter(lambda w: len(w)>1,data2d)))
    # Now we can access the 2d array for the words
    words = data2d[:,0]
    # and the tags
    tags = data2d[:,1]
    # return everything
    return lines, words, tags
# Hope i don't need to explain what this function does :D
def get_most_common_tag(word:str,  print_text = True)->str:
    # using the tags and words as a global variable because of speed
    global tags, words, most_common_tag_in_whole_text
    # Hey Siri, give me all the indices where my word appears
    word_indices = np.where(words==word.lower())
    # Fallback if word appears only once
    if len(word_indices[0]) == 1:
        if print_text: print('No other words found. Taking most common Tag: {}'.format(most_common_tag_in_whole_text))
        return most_common_tag_in_whole_text
    # Hey Google, give me all the tags for that word
    filtered_tags = tags[word_indices]
    # Hey Alexa, give me all the counts for each unique tag
    unique_tags, tag_count = np.unique(filtered_tags, return_counts=True)
    # Hey Cortana, what is the index of the most counted Tag?
    max_idx = np.argmax(tag_count)
    # Hey Bixby, Give me the most common tag and it's probability
    probabilities = tag_count/tag_count.sum()
    most_common_tag = unique_tags[max_idx]
    if print_text: print('Hello Matin, The most common Tag for "{}" is {} with a probability of {:.2%}'.format(word,most_common_tag.upper(),probabilities[max_idx]))
    return most_common_tag

def get_most_common_next_tag(tag:str , print_text = True)->str:
    # using the tags and words as a global variable because of speed
    global tags, words
    # Give me all the indices of the next tag
    next_indices = np.where(tags==tag)[0]+1
    # Remove the last index to not get out of bounds
    next_indices = next_indices[:-1]
    next_tags = tags[next_indices]
    #  Give me all the counts for each unique next tag
    next_tags_unique, next_tags_counts = np.unique(next_tags, return_counts=True)
    # Hey Siri, what is the most counted next Tag and it's probability
    max_idx = np.argmax(next_tags_counts)
    probabilities = next_tags_counts/next_tags_counts.sum()
    most_common_next_tag = next_tags_unique[max_idx]
    if print_text:
        print('The most common Tag after "{}" is {} with a probability of {:.2%}'.format(tag.upper(),most_common_next_tag.upper(),probabilities[max_idx]))
    return most_common_next_tag

def test_data():
    global test_words, most_common_tag_in_whole_text
    cache_words = {}
    cache_tags = {}
    result_tags = []
    for test_word in test_words:
        if test_word in cache_words:
            result_tags.append(cache_words[test_word])
            continue
            
        # Again a fallback
        try:
            tag_of_test_word = get_most_common_tag(test_word , print_text =False)
            if tag_of_test_word in cache_tags:
                result_tags.append(cache_tags[tag_of_test_word])
                continue
            
            next_tag = get_most_common_next_tag(tag_of_test_word,print_text =False )
            cache_tags[tag_of_test_word] = next_tag
            result_tags.append(next_tag)
        except:
            result_tags.append(most_common_tag_in_whole_text)
        
    return result_tags

# Let the games begin
#First parse Train and test file
lines, words, tags= parse_file(file_path)
test_lines, test_words, test_tags= parse_file(test_file_path)

In [181]:
# Let's find the most common words and most common tags
unique_words, words_count = np.unique(words, return_counts=True)
max_idx = np.argmax(words_count)
most_common_word_in_whole_text = unique_words[max_idx]
unique_tags, tags_count = np.unique(tags, return_counts=True)
max_idx = np.argmax(tags_count)
most_common_tag_in_whole_text = unique_tags[max_idx]
# We need this as a fallback if a word only appears once...

In [169]:
# Let's test it

In [170]:
get_most_common_tag('das')

Hello Matin, The most common Tag for "das" is ART with a probability of 84.82%


'art'

In [195]:
get_most_common_next_tag('nn')

The most common Tag after "nn" is APPR with a probability of 19.11%


'appr'

In [184]:
print("Testing data... sorry very slow. Might take 5 Minutes")
result_tags = test_data()

In [194]:
success_rate = np.sum(test_tags == result_tags)/ len(test_tags)

In [ ]:
print('The success rate is {:.2%}'.format(success_rate))